In [1]:
import os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import math

In [2]:
gpus = tf.config.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
BATCH_SIZE = 32
FILTER_SIZE = 3
FILTER_STRIDE = 1 
REGULARIZER_LAMBDA = 0.01
TRAIN_VAL_SPLIT=0.1
TEST_SPLIT_VAL = 0.5

In [4]:
data_path = os.path.join("../data/disease")

In [5]:
train_data, val_data = tf.keras.utils.image_dataset_from_directory(data_path, label_mode="int", batch_size=BATCH_SIZE, seed=100, validation_split=TRAIN_VAL_SPLIT, subset="both")
boundary = math.floor(TEST_SPLIT_VAL*val_data.cardinality().numpy())
test_data = val_data.take(boundary)
val_data = val_data.skip(boundary)
print(f"Using {val_data.cardinality().numpy()*BATCH_SIZE} files for validation.")
print(f"Using {test_data.cardinality().numpy()*BATCH_SIZE} files for testing.")

Found 48117 files belonging to 56 classes.
Using 43306 files for training.
Using 4811 files for validation.
Using 2432 files for validation.
Using 2400 files for testing.


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import splitfolders

In [7]:
SEED = 99
# splitfolders.ratio("../data/disease", output="../data/split", seed=SEED, ratio=(0.9, 0.05, 0.05), group_prefix=None, move=False)

In [8]:
train_augment = ImageDataGenerator(
rotation_range=360, fill_mode="wrap", width_shift_range = 0.1, height_shift_range = 0.1, brightness_range=(0.5, 1.5), zoom_range=0.2, horizontal_flip=True, vertical_flip=True, rescale=1./255
) 
val_test_augment = ImageDataGenerator(rescale=1./255)

train_generator = train_augment.flow_from_directory(directory="../data/split/train", batch_size=32, target_size=(256, 256), color_mode="rgb", class_mode="sparse", seed=SEED)
test_generator = val_test_augment.flow_from_directory(directory="../data/split/test", batch_size=1, target_size=(256, 256), color_mode="rgb", class_mode="sparse", seed=SEED)
val_generator = val_test_augment.flow_from_directory(directory="../data/split/val", batch_size=32, target_size=(256, 256), color_mode="rgb", class_mode="sparse", seed=SEED)

Found 43281 images belonging to 56 classes.
Found 2455 images belonging to 56 classes.
Found 2380 images belonging to 56 classes.


In [9]:
print((train_generator.class_indices.keys()))

dict_keys(['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Cherry___Powdery_mildew', 'Cherry___healthy', 'Citrus___Black_spot', 'Citrus___Canker', 'Citrus___Greening', 'Citrus___Healthy', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___Northern_Leaf_Blight', 'Corn___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Rice___BrownSpot', 'Rice___Healthy', 'Rice___Hispa', 'Rice___LeafBlast', 'Soybean___Bacterial_Pustule', 'Soybean___Frogeye_Leaf_Spot', 'Soybean___Healthy', 'Soybean___Rust', 'Soybean___Sudden_Death_Syndrome', 'Soybean___Target_Leaf_Spot', 'Soybean___Yellow_Mosaic', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Sugarcane___Healthy', 'Sugarcane___Mosaic

In [ ]:
# train_augment_2 = ImageDataGenerator(
# rotation_range=360, fill_mode="wrap", width_shift_range = 0.1, height_shift_range = 0.1, brightness_range=(0.5, 1.5), zoom_range=0.2, horizontal_flip=True, vertical_flip=True, rescale=1./255
# ) 
# val_test_augment_2 = ImageDataGenerator(rescale=1./255)

# train_generator_2 = train_augment.flow_from_directory(directory="../data/split2/train", batch_size=32, target_size=(256, 256), color_mode="rgb", class_mode="sparse", seed=SEED)
# test_generator_2 = val_test_augment.flow_from_directory(directory="../data/split2/test", batch_size=1, target_size=(256, 256), color_mode="rgb", class_mode="sparse", seed=SEED)
# val_generator_2 = val_test_augment.flow_from_directory(directory="../data/split2/val", batch_size=32, target_size=(256, 256), color_mode="rgb", class_mode="sparse", seed=SEED)

In [10]:
mobile_train_generator = train_augment.flow_from_directory(directory="../data/split/train", batch_size=32, target_size=(224, 224), color_mode="rgb", class_mode="sparse", seed=SEED)
mobile_test_generator = val_test_augment.flow_from_directory(directory="../data/split/test", batch_size=1, target_size=(224, 224), color_mode="rgb", class_mode="sparse", seed=SEED)
mobile_val_generator = val_test_augment.flow_from_directory(directory="../data/split/val", batch_size=32, target_size=(224, 224), color_mode="rgb", class_mode="sparse", seed=SEED)

Found 43281 images belonging to 56 classes.
Found 2455 images belonging to 56 classes.
Found 2380 images belonging to 56 classes.


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, Accuracy
from tensorflow.keras.callbacks import EarlyStopping

In [10]:
# resize = tf.keras.layers.Resizing(height=256, width=256, crop_to_aspect_ratio=True)
# flip = tf.keras.layers.RandomFlip("horizontal_and_vertical")
# rotate = tf.keras.layers.RandomRotation(1) # Full 360 degree rotation
# crop = tf.keras.layers.RandomCrop(128, 128) # Doesn't work on Mac for some reason
# zoom = tf.keras.layers.RandomZoom(-0.2, 0.2)
# LABEL_NAMES = train_data.class_names
# NUM_LABELS = len(LABEL_NAMES)
# fig, ax = plt.subplots(ncols=4, nrows=2, figsize=(32, 12))
# train_np_iterator = train_data.as_numpy_iterator()
# batch = train_np_iterator.next()
# for idx, img in enumerate(batch[0][:4]):
#     ax[0][idx].imshow(img.astype(int))
#     ax[0][idx].title.set_text(f"{batch[1][idx]} - {LABEL_NAMES[batch[1][idx]]}")
#     ax[1][idx].imshow(resize(img).numpy().astype(int))
#     ax[1][idx].title.set_text(f"Resized - {batch[1][idx]} - {LABEL_NAMES[batch[1][idx]]}")
#     ax[1][idx].imshow(rotate(img).numpy().astype(int))
#     ax[1][idx].title.set_text(f"Cropped - {batch[1][idx]} - {LABEL_NAMES[batch[1][idx]]}")

## Baseline model - Initial testing

In [37]:
baseline_model = Sequential()
baseline_model.add(Conv2D(1, (3, 3), 1, activation="relu", padding="same", input_shape=(256, 256, 3)))
baseline_model.add(MaxPooling2D(pool_size=(2, 2), strides=2)) # 128
baseline_model.add(Flatten())
baseline_model.add(Dense(128, activation="relu"))
baseline_model.add(Dense(56, activation="softmax")) # multi-class classification

In [38]:
baseline_stopper = EarlyStopping(
    monitor="val_loss",
    patience=2,
    verbose=1,
    mode="min",
    restore_best_weights=True
)

In [39]:
baseline_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [40]:
baseline_model.build((None, 256, 256, 3))

In [41]:
baseline_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 1)       28        
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 1)       0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 16384)             0         
                                                                 
 dense_9 (Dense)             (None, 128)               2097280   
                                                                 
 dense_10 (Dense)            (None, 56)                7224      
                                                                 
Total params: 2104532 (8.03 MB)
Trainable params: 2104532 (8.03 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [32]:
baseline_hist = baseline_model.fit_generator(train_generator, epochs=20, validation_data=val_generator, callbacks=[baseline_stopper])

/var/folders/l0/n_x83b7d1_q6ljq1n9bjz9x40000gn/T/ipykernel_39437/214543348.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  baseline_hist = baseline_model.fit_generator(train_generator, epochs=20, validation_data=val_generator, callbacks=[baseline_stopper])


Epoch 1/20
1353/1353 [==============================] - 523s 386ms/step - loss: 3.3430 - accuracy: 0.1820 - val_loss: 6.2175 - val_accuracy: 0.1832
Epoch 2/20
1353/1353 [==============================] - 511s 377ms/step - loss: 14.6731 - accuracy: 0.1082 - val_loss: 26.1795 - val_accuracy: 0.1008
Epoch 3/20
1353/1353 [==============================] - 509s 376ms/step - loss: 26.9482 - accuracy: 0.1018 - val_loss: 24.6842 - val_accuracy: 0.1113
Epoch 3: early stopping


In [33]:
print(baseline_model.evaluate(test_generator))

2455/2455 [==============================] - 16s 6ms/step - loss: 6.2231 - accuracy: 0.1882
[6.223121643066406, 0.188187375664711]


### **So, accuracy for baseline model is 18.9%**

In [32]:
import tensorflow_hub as hub
from keras_tuner.tuners import RandomSearch
from keras.optimizers.legacy import Adam
from keras.layers import Reshape

## Mobile Net Model

In [33]:
def mobile_net(hp):
    url = "https://www.kaggle.com/models/google/mobilenet-v3/frameworks/TensorFlow2/variations/large-075-224-feature-vector/versions/1"
    base_mobile_model = hub.KerasLayer(url)
    base_mobile_model.trainable = False
    new_mobile_model = Sequential([
        base_mobile_model,
        Dense(256, activation="relu"),
        Dense(128, activation="relu"),
        Dense(56, activation="softmax")
    ])

    new_mobile_model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(learning_rate=hp.Float("learning_rate", min_value=1e-5, max_value=1e-1, step=10, sampling="log")), metrics=["accuracy"])
    return new_mobile_model

In [34]:
tuner_mobile_net = RandomSearch(
    mobile_net,
    objective="val_accuracy",
    max_trials=10000, # just default to max number of configs
    executions_per_trial=1,
    directory="./models/final",
    project_name="Mobile_Net_Final"
)

Reloading Tuner from ./models/final/Mobile_Net_Final/tuner0.json


In [42]:
tuner_mobile_net.search_space_summary()

Search space summary
Default search space size: 1
learning_rate (Float)
{'default': 1e-05, 'conditions': [], 'min_value': 1e-05, 'max_value': 0.1, 'step': 10, 'sampling': 'log'}


In [43]:
tuner_mobile_net.search(mobile_train_generator,callbacks=[baseline_stopper], epochs=27, validation_data=mobile_val_generator) # Can use baseline_stopper for stopper

In [44]:
mobile_hps = tuner_mobile_net.oracle.get_best_trials(num_trials=1)[0].hyperparameters
final_mobile_net = mobile_net(mobile_hps)
print(final_mobile_net.optimizer.get_config())

{'name': 'Adam', 'learning_rate': 0.0001, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}


In [47]:
final_mobile_net.fit(mobile_train_generator, epochs=27, callbacks=[baseline_stopper], validation_data=mobile_val_generator)

Epoch 1/27
1353/1353 [==============================] - 484s 358ms/step - loss: 0.2444 - accuracy: 0.9124 - val_loss: 0.2600 - val_accuracy: 0.9067
Epoch 2/27
1353/1353 [==============================] - 552s 408ms/step - loss: 0.2314 - accuracy: 0.9175 - val_loss: 0.2513 - val_accuracy: 0.9164
Epoch 3/27
1353/1353 [==============================] - 522s 386ms/step - loss: 0.2252 - accuracy: 0.9193 - val_loss: 0.2536 - val_accuracy: 0.9101
Epoch 4/27
1353/1353 [==============================] - 457s 338ms/step - loss: 0.2207 - accuracy: 0.9204 - val_loss: 0.2465 - val_accuracy: 0.9139
Epoch 5/27
1353/1353 [==============================] - 447s 331ms/step - loss: 0.2162 - accuracy: 0.9220 - val_loss: 0.2324 - val_accuracy: 0.9164
Epoch 6/27
1353/1353 [==============================] - 441s 326ms/step - loss: 0.2103 - accuracy: 0.9226 - val_loss: 0.2348 - val_accuracy: 0.9197
Epoch 7/27
1353/1353 [==============================] - 446s 330ms/step - loss: 0.2050 - accuracy: 0.9255 - val_

In [48]:
final_mobile_net.evaluate(mobile_test_generator)

2455/2455 [==============================] - 82s 33ms/step - loss: 0.2501 - accuracy: 0.9096


[0.25006043910980225, 0.9095723032951355]

In [49]:
final_mobile_net.save("./models/final/SAVED_MODELS/Mobile_Net/Mobile_Net.keras")

In [50]:
final_mobile_net.save("./models/final/SAVED_MODELS/Mobile_Net/SavedModelFormat")

INFO:tensorflow:Assets written to: ./models/final/SAVED_MODELS/Mobile_Net/SavedModelFormat/assets


INFO:tensorflow:Assets written to: ./models/final/SAVED_MODELS/Mobile_Net/SavedModelFormat/assets


In [51]:
final_mobile_net.save("./models/final/SAVED_MODELS/Mobile_Net/Mobile_Net.h5")

/Users/tejas/miniconda3/envs/dl/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### **So, accuracy for MobileNet model is 91%**
In my Rise video, I said 93%. The difference in accuracy is because I continued work on my Rise project after submission and trained the model again after submitting, so there was a slight change in accuracy.

## EfficientNet Model

In [13]:
def efficient_net(hp):
    url = "https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet21k-m-feature-vector/versions/2"
    base_efficientnet_model = hub.KerasLayer(url, input_shape=(256, 256, 3))
    base_efficientnet_model.trainable = False
    new_efficientnet_model = Sequential([
        base_efficientnet_model,
        Dense(256, activation="relu"),
        Dense(128, activation="relu"),
        Dense(56, activation="softmax")
    ])
    new_efficientnet_model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(learning_rate=hp.Float("learning_rate", min_value=1e-5, max_value=1e-1, step=10, sampling="log")), metrics=["accuracy"])
    return new_efficientnet_model


In [14]:
tuner_efficient_net = RandomSearch(
    efficient_net,
    objective="val_accuracy",
    max_trials=10000, # just default to max number of configs
    executions_per_trial=1,
    directory="./models/final",
    project_name="Efficient_Net_Final"
)

In [15]:
efficient_stopper = EarlyStopping(
    monitor="val_loss",
    patience=2,
    verbose=1,
    mode="min",
    restore_best_weights=True
)

In [16]:
tuner_efficient_net.search(train_generator, callbacks=[efficient_stopper], epochs=27, validation_data=val_generator)

Trial 5 Complete [00h 28m 48s]
val_accuracy: 0.8415966629981995

Best val_accuracy So Far: 0.9474790096282959
Total elapsed time: 07h 13m 33s


In [18]:
efficient_hps = tuner_efficient_net.oracle.get_best_trials(num_trials=1)[0].hyperparameters
final_efficient_net = efficient_net(efficient_hps)
print(final_efficient_net.optimizer.get_config())

{'name': 'Adam', 'learning_rate': 0.0001, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}


In [19]:
final_efficient_net.fit(train_generator, epochs=27, callbacks=[efficient_stopper], validation_data=val_generator)

Epoch 1/27
1353/1353 [==============================] - 750s 541ms/step - loss: 0.8302 - accuracy: 0.7713 - val_loss: 0.3165 - val_accuracy: 0.8962
Epoch 2/27
1353/1353 [==============================] - 680s 503ms/step - loss: 0.2592 - accuracy: 0.9113 - val_loss: 0.2141 - val_accuracy: 0.9248
Epoch 3/27
1353/1353 [==============================] - 572s 422ms/step - loss: 0.1988 - accuracy: 0.9315 - val_loss: 0.1968 - val_accuracy: 0.9303
Epoch 4/27
1353/1353 [==============================] - 563s 416ms/step - loss: 0.1705 - accuracy: 0.9414 - val_loss: 0.1871 - val_accuracy: 0.9307
Epoch 5/27
1353/1353 [==============================] - 575s 425ms/step - loss: 0.1548 - accuracy: 0.9442 - val_loss: 0.1510 - val_accuracy: 0.9433
Epoch 6/27
1353/1353 [==============================] - 564s 417ms/step - loss: 0.1412 - accuracy: 0.9500 - val_loss: 0.1601 - val_accuracy: 0.9433
Epoch 7/27
1353/1353 [==============================] - 566s 418ms/step - loss: 0.1329 - accuracy: 0.9525 - val_

In [21]:
final_efficient_net.evaluate(test_generator)

2455/2455 [==============================] - 201s 82ms/step - loss: 0.1488 - accuracy: 0.9495


[0.14879626035690308, 0.9494908452033997]

In [26]:
final_efficient_net.save("./models/final/SAVED_MODELS/Efficient_Net/Efficient_Net.keras")

In [27]:
final_efficient_net.save("./models/final/SAVED_MODELS/Efficient_Net/SavedModelFormat")

INFO:tensorflow:Assets written to: ./models/final/SAVED_MODELS/Efficient_Net/SavedModelFormat/assets


INFO:tensorflow:Assets written to: ./models/final/SAVED_MODELS/Efficient_Net/SavedModelFormat/assets


In [28]:
final_efficient_net.save("./models/final/SAVED_MODELS/Efficient_Net/Efficient_Net.h5")

/Users/tejas/miniconda3/envs/dl/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### **So, accuracy for EfficientNet model is 94.9%**
In my Rise video, I said 96%. The difference in accuracy is because I continued work on my Rise project after submission and trained the model again after submitting, so there was a slight change in accuracy.